# Utility functions for dataset

In [1]:
import zipfile
import shutil

# Function to unzip a dataset
def unzip_dataset(zip_path, extract_to='.'):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Function to delete folders and their contents
def delete_folder_and_contents(folder_path='.'):
    # Define the paths to the folders and file to delete
    images_folder = 'images/'
    gt_images_folder = 'gt-images/'
    query_image = 'query.jpg'

    # Delete the folders and file non-recursively
    if os.path.exists(images_folder) and os.path.isdir(images_folder):
        for root, dirs, files in os.walk(images_folder, topdown=False):
            for name in files:
                os.remove(os.path.join(root, name))
            for name in dirs:
                os.rmdir(os.path.join(root, name))
        os.rmdir(images_folder)

    if os.path.exists(gt_images_folder) and os.path.isdir(gt_images_folder):
        for root, dirs, files in os.walk(gt_images_folder, topdown=False):
            for name in files:
                os.remove(os.path.join(root, name))
            for name in dirs:
                os.rmdir(os.path.join(root, name))
        os.rmdir(gt_images_folder)

    if os.path.exists(query_image):
        os.remove(query_image)

# Install Dependencies

In [2]:
# Install necessary dependencies
!pip install opencv-python-headless
!pip install numpy
!pip install pillow
!pip install torch torchvision
!pip install scikit-learn
!pip install transformers
!pip install imagehash
!pip install tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.5/296.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 53.1 MB/s eta 0:00:00


# Import Libraries

In [3]:
import os
import cv2
import numpy as np
from PIL import Image
import torch
from torchvision import models, transforms
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, accuracy_score
from transformers import ViTFeatureExtractor, ViTModel, CLIPProcessor, CLIPModel
from concurrent.futures import ThreadPoolExecutor
import imagehash
import time
from tqdm import tqdm


# Utility Functions

In [4]:
# --------------------------- Utility Functions --------------------------- #
def validate_image_paths(image_paths):
    """Validate that the provided image paths exist and are of correct format."""
    valid_images = []
    for path in image_paths:
        if os.path.exists(path) and path.lower().endswith(('.jpg', '.jpeg', '.png')):
            valid_images.append(path)
        else:
            print(f"Invalid image path or format: {path}")
    return valid_images

def load_ground_truth(gt_folder):
    """Load ground truth images from a folder."""
    ground_truth_images = []
    for image_file in os.listdir(gt_folder):
        if image_file.endswith(('.jpg', '.png', '.jpeg')):
            image_path = os.path.join(gt_folder, image_file)
            ground_truth_images.append(image_path)
    return ground_truth_images


# ORB Similarity

In [5]:
# --------------------------- ORB Similarity --------------------------- #
def orb_similarity(query_objects, dataset_objects, nfeatures=5000, match_distance_threshold=50):
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    orb = cv2.ORB_create(nfeatures=nfeatures)

    def orb_features(image_path):
        try:
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                print(f"Failed to load image: {image_path}")
                return None, None
            kp, des = orb.detectAndCompute(image, None)
            return kp, des
        except Exception as e:
            print(f"Error processing {image_path}: {e}")
            return None, None

    with ThreadPoolExecutor() as executor:
        query_descriptors = list(executor.map(orb_features, query_objects))
        dataset_descriptors = list(executor.map(orb_features, dataset_objects))

    results = []
    for i, (query_kp, query_des) in enumerate(query_descriptors):
        if query_des is None:
            continue
        for j, (dataset_kp, dataset_des) in enumerate(dataset_descriptors):
            if dataset_des is None:
                continue
            matches = bf.match(query_des, dataset_des)
            good_matches = [m for m in matches if m.distance < match_distance_threshold]
            similarity = len(good_matches)

            # Normalize the similarity score
            max_possible_matches = min(len(query_kp), len(dataset_kp))
            normalized_similarity = similarity / max_possible_matches if max_possible_matches > 0 else 0

            results.append((query_objects[i], dataset_objects[j], normalized_similarity))

    results.sort(key=lambda x: x[2], reverse=True)
    return results


# ViT Similarity

In [6]:
# --------------------------- ViT Similarity --------------------------- #
def vit_similarity(query_objects, dataset_objects, batch_size=16):
    """
    Calculate similarity between query and dataset images using ViT.

    Args:
        query_objects (list): List of file paths to query images.
        dataset_objects (list): List of file paths to dataset images.
        batch_size (int): Number of images to process in a batch.

    Returns:
        list: Sorted list of (query_path, dataset_path, similarity) tuples.
    """
    feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
    vit_model = ViTModel.from_pretrained('google/vit-base-patch16-224')

    def extract_vit_features(image_path):
        """
        Extract ViT features for a single image.

        Args:
            image_path (str): Path to the image file.

        Returns:
            tuple: (image_path, feature_array)
        """
        try:
            image = Image.open(image_path).convert('RGB')
            inputs = feature_extractor(images=image, return_tensors="pt")
            with torch.no_grad():
                outputs = vit_model(**inputs)
            return image_path, outputs.pooler_output.squeeze().numpy()
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
            return image_path, None

    def process_images_in_batches(image_paths):
        """
        Process a list of images in batches.

        Args:
            image_paths (list): List of image file paths.

        Returns:
            dict: Dictionary mapping image paths to their extracted features.
        """
        features = {}
        for i in tqdm(range(0, len(image_paths), batch_size), desc="Extracting features"):
            batch = image_paths[i:i + batch_size]
            with ThreadPoolExecutor() as executor:
                results = list(executor.map(extract_vit_features, batch))
            for image_path, feature in results:
                if feature is not None:
                    features[image_path] = feature
        return features

    # Precompute features for query and dataset images
    print("Precomputing query features...")
    query_features = process_images_in_batches(query_objects)

    print("Precomputing dataset features...")
    dataset_features = process_images_in_batches(dataset_objects)

    # Compute similarity between query and dataset images
    print("Computing similarities...")
    results = []
    for query_path, query_feature in tqdm(query_features.items(), desc="Matching queries"):
        for dataset_path, dataset_feature in dataset_features.items():
            try:
                similarity = cosine_similarity([query_feature], [dataset_feature])[0][0]
                results.append((query_path, dataset_path, similarity))
            except Exception as e:
                print(f"Error comparing {query_path} with {dataset_path}: {e}")

    # Sort results by similarity in descending order
    results.sort(key=lambda x: x[2], reverse=True)
    return results


# CNN/VGG16 Similarity

In [7]:
# --------------------------- CNN/VGG16 Similarity --------------------------- #
def cnn_similarity(query_objects, dataset_objects, batch_size=16):
    """
    Calculate similarity between query and dataset images using ResNet50.

    Args:
        query_objects (list): List of file paths to query images.
        dataset_objects (list): List of file paths to dataset images.
        batch_size (int): Number of images to process in a batch.

    Returns:
        list: Sorted list of (query_path, dataset_path, similarity) tuples.
    """
    # Load the pre-trained ResNet50 model
    resnet50_model = models.resnet50(pretrained=True).eval()

    # Modify the model to output features instead of classification scores
    resnet50_model = torch.nn.Sequential(*(list(resnet50_model.children())[:-1]))  # Remove the final classification layer

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    def extract_cnn_features(image_paths):
        """
        Extract CNN features for a batch of images.

        Args:
            image_paths (list): List of image file paths.

        Returns:
            dict: Dictionary mapping image paths to their extracted features.
        """
        features = {}
        try:
            images = [transform(Image.open(image_path).convert('RGB')).unsqueeze(0) for image_path in image_paths]
            images = torch.cat(images, dim=0)
            with torch.no_grad():
                feature_batch = resnet50_model(images).flatten(1).cpu().numpy()
            for image_path, feature in zip(image_paths, feature_batch):
                features[image_path] = feature
        except Exception as e:
            print(f"Error processing batch: {e}")
        return features

    def batch_process(image_paths, batch_size):
        """
        Split image paths into batches.

        Args:
            image_paths (list): List of image file paths.
            batch_size (int): Size of each batch.

        Returns:
            list: List of batches (sublists).
        """
        return [image_paths[i:i + batch_size] for i in range(0, len(image_paths), batch_size)]

    # Precompute dataset features for efficiency
    print("Precomputing dataset features...")
    dataset_features = {}
    for batch in batch_process(dataset_objects, batch_size):
        dataset_features.update(extract_cnn_features(batch))

    # Compute similarity between each query and dataset image
    results = []
    for query_path in query_objects:
        query_features = extract_cnn_features([query_path])
        if not query_features:
            print(f"Skipping query image {query_path} due to errors.")
            continue

        query_feature = list(query_features.values())[0]
        for dataset_path, dataset_feature in dataset_features.items():
            try:
                similarity = cosine_similarity([query_feature], [dataset_feature])[0][0]
                results.append((query_path, dataset_path, similarity))
            except Exception as e:
                print(f"Error comparing {query_path} with {dataset_path}: {e}")

    # Sort results by similarity in descending order
    results.sort(key=lambda x: x[2], reverse=True)
    return results


# Perceptual Hashing

In [8]:
# --------------------------- Perceptual Hashing --------------------------- #
def phash_similarity(query_objects, dataset_objects, similarity_metric="hamming"):
    """
    Calculate Perceptual Hashing similarity between query and dataset images.

    Args:
        query_objects (list): List of file paths to query images.
        dataset_objects (list): List of file paths to dataset images.
        similarity_metric (str): Similarity metric to use ("hamming" by default).
                                 Currently supports Hamming distance.

    Returns:
        list: Sorted list of (query_path, dataset_path, similarity) tuples.
    """
    # Precompute dataset hashes for efficiency
    dataset_hashes = {}
    for dataset_path in dataset_objects:
        try:
            dataset_hashes[dataset_path] = imagehash.phash(Image.open(dataset_path))
        except Exception as e:
            print(f"Error processing dataset image {dataset_path}: {e}")
            continue

    results = []
    for query_path in query_objects:
        try:
            query_hash = imagehash.phash(Image.open(query_path))
        except Exception as e:
            print(f"Error processing query image {query_path}: {e}")
            continue

        for dataset_path, dataset_hash in dataset_hashes.items():
            try:
                # Calculate similarity based on chosen metric
                if similarity_metric == "hamming":
                    similarity = -abs(query_hash - dataset_hash)  # Negative Hamming distance
                else:
                    raise ValueError(f"Unsupported similarity metric: {similarity_metric}")
                results.append((query_path, dataset_path, similarity))
            except Exception as e:
                print(f"Error comparing {query_path} with {dataset_path}: {e}")
                continue

    # Sort results by similarity (descending)
    results.sort(key=lambda x: x[2], reverse=True)
    return results


# CLIP Similarity

In [9]:
# --------------------------- CLIP Similarity --------------------------- #
def clip_similarity(query_objects, dataset_objects, batch_size=8):
    """
    Calculate CLIP-based similarity between query and dataset images.

    Args:
        query_objects (list): List of file paths to query images.
        dataset_objects (list): List of file paths to dataset images.
        batch_size (int): Number of images to process in a batch (for efficiency).

    Returns:
        list: Sorted list of (query_path, dataset_path, similarity) tuples.
    """
    # Load the latest CLIP model and processor
    clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")
    clip_model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")

    def extract_clip_features(image_path):
        """
        Extract features for a single image using CLIP.

        Args:
            image_path (str): Path to the image.

        Returns:
            np.ndarray or None: Extracted features or None if an error occurs.
        """
        try:
            image = Image.open(image_path).convert('RGB')
            inputs = clip_processor(images=image, return_tensors="pt")
            with torch.no_grad():
                outputs = clip_model.get_image_features(**inputs)
            return outputs.squeeze().numpy()
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")
            return None

    def extract_features_batch(image_paths):
        """
        Extract features for a batch of images.

        Args:
            image_paths (list): List of image file paths.

        Returns:
            dict: Dictionary of image paths to their corresponding features.
        """
        features = {}
        for image_path in image_paths:
            feature = extract_clip_features(image_path)
            if feature is not None:
                features[image_path] = feature
        return features

    # Precompute dataset features for efficiency
    print("Precomputing dataset features...")
    dataset_features = extract_features_batch(dataset_objects)

    results = []
    for query_path in query_objects:
        query_features = extract_clip_features(query_path)
        if query_features is None:
            continue

        for dataset_path, dataset_feature in dataset_features.items():
            try:
                similarity = cosine_similarity([query_features], [dataset_feature])[0][0]
                results.append((query_path, dataset_path, similarity))
            except Exception as e:
                print(f"Error comparing {query_path} with {dataset_path}: {e}")
                continue

    # Sort results by similarity (descending)
    results.sort(key=lambda x: x[2], reverse=True)
    return results


# Performance Analysis

In [10]:
# --------------------------- Performance Analysis --------------------------- #
def analyze_performance(method, method_name, query_objects, dataset_objects, ground_truth):
    start_time = time.time()
    try:
        results = method(query_objects, dataset_objects)
    except Exception as e:
        print(f"Error in {method_name}: {e}")
        return
    end_time = time.time()

    print(f"\n--- {method_name} Results ---")
    if not results or len(results[0]) < 3:
        print(f"No valid results returned by {method_name}.")
        return

    # Sort results by query and dataset file names
    results.sort(key=lambda x: (x[0], x[1]))

    # Display top 5 results
    for result in results:
        print(f"Query: {result[0]}, Match: {result[1]}, Score: {result[2]:.4f}")

    # Extract scores
    scores = [result[2] for result in results]

    # Use the median of the scores as the threshold
    optimal_threshold = np.mean(scores)
    print("Optimal Threshold:", optimal_threshold)

    # Create y_pred based on the optimal threshold
    y_pred = [1 if score >= optimal_threshold else 0 for score in scores]

    print("y_pred:", y_pred)

    y_true = [1 if 'gt-'+i[1] in ground_truth else 0 for i in results ]
    print("y_true:", y_true)

    # Calculate precision, recall, and retrieval accuracy
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    retrieval_accuracy = accuracy_score(y_true, y_pred)

    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"Retrieval Accuracy: {retrieval_accuracy:.4f}")
    print(f"Time Taken: {end_time - start_time:.2f} seconds")


In [11]:
def get_analysis(zip_path):

        unzip_dataset(zip_path)

        # Define the paths to the query image, dataset folder, and ground truth folder
        query_image = 'query.jpg'
        dataset_folder = 'images/'
        gt_folder = 'gt-images/'

        # Validate image paths
        query_objects = validate_image_paths([query_image])
        dataset_objects = validate_image_paths([os.path.join(dataset_folder, f) for f in os.listdir(dataset_folder)])
        ground_truth = load_ground_truth(gt_folder)

        # Analyze performance for each method
        print(f"\n--- Results for {zip_path} ---")
        analyze_performance(orb_similarity, "ORB Similarity", query_objects, dataset_objects, ground_truth)
        analyze_performance(vit_similarity, "ViT Similarity", query_objects, dataset_objects, ground_truth)
        analyze_performance(cnn_similarity, "CNN Similarity", query_objects, dataset_objects, ground_truth)
        analyze_performance(phash_similarity, "Perceptual Hashing", query_objects, dataset_objects, ground_truth)
        analyze_performance(clip_similarity, "CLIP", query_objects, dataset_objects, ground_truth)

        # Delete the extracted dataset folder
        delete_folder_and_contents()

# Analysis of Dataset1 to find dogs

In [13]:
zipped_datasets = ['dataset1(dog).zip', 'dataset2(samsung).zip', 'dataset3(doraemon).zip']

get_analysis(zipped_datasets[0])


--- Results for dataset1(dog).zip ---

--- ORB Similarity Results ---
Query: query.jpg, Match: images/calf1.jpeg, Score: 0.0608
Query: query.jpg, Match: images/cat1.jpg, Score: 0.0858
Query: query.jpg, Match: images/cat2.jpg, Score: 0.0527
Query: query.jpg, Match: images/cat3.jpg, Score: 0.1122
Query: query.jpg, Match: images/cat4.jpg, Score: 0.0882
Query: query.jpg, Match: images/cat5.jpeg, Score: 0.0223
Query: query.jpg, Match: images/dog1.jpg, Score: 0.1220
Query: query.jpg, Match: images/dog2.jpeg, Score: 0.0471
Query: query.jpg, Match: images/dog3.jpeg, Score: 0.0458
Query: query.jpg, Match: images/dog4.jpeg, Score: 0.0500
Query: query.jpg, Match: images/dog5.jpeg, Score: 0.0935
Query: query.jpg, Match: images/dog6.jpeg, Score: 0.0456
Query: query.jpg, Match: images/dog7.jpeg, Score: 0.0268
Query: query.jpg, Match: images/dog8.jpeg, Score: 0.0813
Query: query.jpg, Match: images/panther1.jpg, Score: 0.0796
Query: query.jpg, Match: images/panther2.jpeg, Score: 0.0237
Query: query.j

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.52k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

Precomputing dataset features...

--- CLIP Results ---
Query: query.jpg, Match: images/calf1.jpeg, Score: 0.6253
Query: query.jpg, Match: images/cat1.jpg, Score: 0.7356
Query: query.jpg, Match: images/cat2.jpg, Score: 0.8129
Query: query.jpg, Match: images/cat3.jpg, Score: 0.8330
Query: query.jpg, Match: images/cat4.jpg, Score: 0.7201
Query: query.jpg, Match: images/cat5.jpeg, Score: 0.7055
Query: query.jpg, Match: images/dog1.jpg, Score: 0.7993
Query: query.jpg, Match: images/dog2.jpeg, Score: 0.8145
Query: query.jpg, Match: images/dog3.jpeg, Score: 0.8029
Query: query.jpg, Match: images/dog4.jpeg, Score: 0.8347
Query: query.jpg, Match: images/dog5.jpeg, Score: 0.7107
Query: query.jpg, Match: images/dog6.jpeg, Score: 0.7999
Query: query.jpg, Match: images/dog7.jpeg, Score: 0.7887
Query: query.jpg, Match: images/dog8.jpeg, Score: 0.7512
Query: query.jpg, Match: images/panther1.jpg, Score: 0.6836
Query: query.jpg, Match: images/panther2.jpeg, Score: 0.6231
Query: query.jpg, Match: image

# Analysis of Dataset 2 to find Samsung M04

In [14]:
get_analysis(zipped_datasets[1])


--- Results for dataset2(samsung).zip ---

--- ORB Similarity Results ---
Query: query.jpg, Match: images/iphone1.jpeg, Score: 0.2892
Query: query.jpg, Match: images/iphone2.jpeg, Score: 0.2388
Query: query.jpg, Match: images/iphone3.jpeg, Score: 0.3016
Query: query.jpg, Match: images/iphone4.jpeg, Score: 0.2029
Query: query.jpg, Match: images/iphone5.jpeg, Score: 0.1768
Query: query.jpg, Match: images/iphone6.jpg, Score: 0.2000
Query: query.jpg, Match: images/other1.jpeg, Score: 0.2887
Query: query.jpg, Match: images/other2.jpg, Score: 0.0052
Query: query.jpg, Match: images/other3.jpeg, Score: 0.2796
Query: query.jpg, Match: images/other4.jpeg, Score: 0.3782
Query: query.jpg, Match: images/samsung1.jpg, Score: 0.0711
Query: query.jpg, Match: images/samsung2.jpeg, Score: 0.3358
Query: query.jpg, Match: images/samsung3.jpeg, Score: 0.1486
Query: query.jpg, Match: images/samsung4.jpg, Score: 0.4079
Query: query.jpg, Match: images/samsung5.jpg, Score: 0.0242
Optimal Threshold: 0.22322972

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precomputing query features...


Extracting features: 100%|██████████| 1/1 [00:00<00:00,  1.45it/s]


Precomputing dataset features...


Extracting features: 100%|██████████| 1/1 [00:10<00:00, 10.63s/it]


Computing similarities...


Matching queries: 100%|██████████| 1/1 [00:00<00:00, 59.03it/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



--- ViT Similarity Results ---
Query: query.jpg, Match: images/iphone1.jpeg, Score: 0.5434
Query: query.jpg, Match: images/iphone2.jpeg, Score: 0.4330
Query: query.jpg, Match: images/iphone3.jpeg, Score: 0.4416
Query: query.jpg, Match: images/iphone4.jpeg, Score: 0.6180
Query: query.jpg, Match: images/iphone5.jpeg, Score: 0.6113
Query: query.jpg, Match: images/iphone6.jpg, Score: 0.6761
Query: query.jpg, Match: images/other1.jpeg, Score: 0.5970
Query: query.jpg, Match: images/other2.jpg, Score: 0.6209
Query: query.jpg, Match: images/other3.jpeg, Score: 0.4655
Query: query.jpg, Match: images/other4.jpeg, Score: 0.6416
Query: query.jpg, Match: images/samsung1.jpg, Score: 0.6166
Query: query.jpg, Match: images/samsung2.jpeg, Score: 0.7682
Query: query.jpg, Match: images/samsung3.jpeg, Score: 0.5615
Query: query.jpg, Match: images/samsung4.jpg, Score: 0.8535
Query: query.jpg, Match: images/samsung5.jpg, Score: 0.6290
Optimal Threshold: 0.6051414
y_pred: [0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1

# Analysis of Dataset 3 to find Doraemon

In [15]:
get_analysis(zipped_datasets[2])


--- Results for dataset3(doraemon).zip ---

--- ORB Similarity Results ---
Query: query.jpg, Match: images/circles.jpg, Score: 0.0474
Query: query.jpg, Match: images/doraemon-cartoon-japanese-free-vector.jpg, Score: 0.1307
Query: query.jpg, Match: images/doraemon1.jpg, Score: 0.0973
Query: query.jpg, Match: images/doraemon2.jpg, Score: 0.0856
Query: query.jpg, Match: images/doraemon3.jpeg, Score: 0.1725
Query: query.jpg, Match: images/doraemon4.jpeg, Score: 0.0241
Query: query.jpg, Match: images/dorami.jpeg, Score: 0.0669
Query: query.jpg, Match: images/pokemon.jpg, Score: 0.0856
Query: query.jpg, Match: images/pokemon1.jpeg, Score: 0.0297
Optimal Threshold: 0.0822048948185477
y_pred: [0, 1, 1, 1, 1, 0, 0, 1, 0]
y_true: [0, 1, 1, 1, 1, 1, 0, 0, 0]
Precision: 0.8000
Recall: 0.8000
Retrieval Accuracy: 0.7778
Time Taken: 1.06 seconds


/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precomputing query features...


Extracting features: 100%|██████████| 1/1 [00:00<00:00,  1.49it/s]


Precomputing dataset features...


Extracting features: 100%|██████████| 1/1 [00:06<00:00,  6.70s/it]


Computing similarities...


Matching queries: 100%|██████████| 1/1 [00:00<00:00, 68.51it/s]
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



--- ViT Similarity Results ---
Query: query.jpg, Match: images/circles.jpg, Score: 0.2598
Query: query.jpg, Match: images/doraemon-cartoon-japanese-free-vector.jpg, Score: 0.5773
Query: query.jpg, Match: images/doraemon1.jpg, Score: 0.3409
Query: query.jpg, Match: images/doraemon2.jpg, Score: 0.5506
Query: query.jpg, Match: images/doraemon3.jpeg, Score: 0.4835
Query: query.jpg, Match: images/doraemon4.jpeg, Score: 0.5457
Query: query.jpg, Match: images/dorami.jpeg, Score: 0.7474
Query: query.jpg, Match: images/pokemon.jpg, Score: 0.4746
Query: query.jpg, Match: images/pokemon1.jpeg, Score: 0.2551
Optimal Threshold: 0.47053993
y_pred: [0, 1, 0, 1, 1, 1, 1, 1, 0]
y_true: [0, 1, 1, 1, 1, 1, 0, 0, 0]
Precision: 0.6667
Recall: 0.8000
Retrieval Accuracy: 0.6667
Time Taken: 7.73 seconds
Precomputing dataset features...

--- CNN Similarity Results ---
Query: query.jpg, Match: images/circles.jpg, Score: 0.7226
Query: query.jpg, Match: images/doraemon-cartoon-japanese-free-vector.jpg, Score: 0.